[link text](https://)bitsandbytes==0.41.2.post2

In [19]:
!pip install accelerate peft bitsandbytes==0.41.2.post2 trl transformers==4.35.2 evaluate rouge_score np

In [ ]:
!pip install transformers transformers[sentencepiece] sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
en_to_el_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-el")
en_to_el_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-el")

gr_to_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-grk-en")
gr_to_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-grk-en")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to acc

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/787k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/441k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/802k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/225M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import Dataset, DatasetDict

# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model
new_model = "llama-2-7b-chat-processes"

# New instruction dataset
from google.colab import drive

drive.mount('/content/drive')
import pandas as pd

# Create a sample Pandas DataFrame
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DATASET/MITOS-EN-EL-QAS.csv', delimiter =',')

# Convert Pandas DataFrame to Hugging Face Dataset and split to train and test
datasets = Dataset.from_pandas(df).train_test_split(test_size=0.2)

dataset = datasets["train"]
eval_dataset = datasets["test"]

# Creation of 4-bit quantization with NF4 type configuration using BitsAndBytes
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Loading model using 4-bit precision with the compute dtype "float16" from Hugging Face for faster training
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Loading tokenizer from Hugginface and set padding_side to “right” to fix the issue with fp16
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# PEFT parameters
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Training parameters
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Model fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)


trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

logging.set_verbosity(logging.CRITICAL)


# SAVE TEST DATA IN EXCEL FILE

import openpyxl

# Create an Excel workbook and add a worksheet
workbook = openpyxl.Workbook()
worksheet = workbook.active

# Add headers to the worksheet
worksheet.append(["text", "text_el"])

for data in eval_dataset:
  # Append the reference and prediction to the Excel file
  worksheet.append([data["text"], data['text_el']])

  # Save the Excel workbook
  workbook.save("test_data.xlsx")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1697: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1697: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/l

Map:   0%|          | 0/755 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
input_text_el = "Ποιες είναι οι προϋποθέσεις για την έκδοση της άδειας μεταφορέα ζώντων ζώων τύπου ΙΙ;"
print('input_text_el: ', input_text_el)

#transformer el to en
encoded_input = gr_to_en_tokenizer(input_text_el, return_tensors="pt", padding=True)
output = gr_to_en_model.generate(**encoded_input)
input_text_en = gr_to_en_tokenizer.batch_decode(output, skip_special_tokens=True)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(f"<s>[INST] {input_text_en} [/INST]")
result = result[0]['generated_text']

#transformer en to el
encoded_input = en_to_el_tokenizer(result, return_tensors="pt", padding=True)
output = en_to_el_model.generate(**encoded_input)
output_text_el = en_to_el_tokenizer.batch_decode(output, skip_special_tokens=True)
print('output_text_el: ', output_text_el)

input_text_el:  Ποιες είναι οι προϋποθέσεις για την έκδοση της άδειας μεταφορέα ζώντων ζώων τύπου ΙΙ;


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
try:
  input_text_el = "Βήματα της διαδικασίας έκδοσης άδειας διακίνησης αιγών και προβάτων"
  print('input_text_el: ', input_text_el)

  #transformer el to en
  encoded_input = gr_to_en_tokenizer(input_text_el, return_tensors="pt", padding=True)
  output = gr_to_en_model.generate(**encoded_input)
  input_text_en = gr_to_en_tokenizer.batch_decode(output, skip_special_tokens=True)

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
  result = pipe(f"<s>[INST] {input_text_en} [/INST]")
  result = result[0]['generated_text']
  print('result: ', result)

  #transformer en to el
  encoded_input = en_to_el_tokenizer(result, return_tensors="pt", padding=True)
  output = en_to_el_model.generate(**encoded_input)
  output_text_el = en_to_el_tokenizer.batch_decode(output, skip_special_tokens=True)
  print('output_text_el: ', output_text_el)
except Exception as e:
  print(e)


input_text_el:  Βήματα της διαδικασίας έκδοσης άδειας διακίνησης αιγών και προβάτων


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


result:  <s>[INST] ['Steps in the authorisation procedure for the movement of goats and sheep'] [/INST]  The authorisation procedure for the movement of goats and sheep may vary depending on the country or region, but here are some general steps that are commonly followed:
 nobody

1. Obtain a movement permit: Before moving goats or sheep to a new location, you will need to obtain a movement permit from the relevant agricultural authority. This permit is usually issued after you have provided information about the animals, including their origin, destination, and any health status.

2. Test for diseases: Before moving goats or sheep, you may be required to test them for diseases such as brucellosis, tuberculosis, and anthrax. The tests may be conducted by a veterinarian or at a government-approved testing station.

3. Vaccinate the animals: Depending on the location and the time of year, you may be required to vaccinate your goats or sheep against certain diseases. This is an important

In [ ]:
try:
  input_text_el = "Ποια είναι τα βήματα της διαδικασίας αλλαγής στοιχείων επικοινωνίας αλλοδαπών;"
  print('input_text_el: ', input_text_el)

  #transformer el to en
  encoded_input = gr_to_en_tokenizer(input_text_el, return_tensors="pt", padding=True)
  output = gr_to_en_model.generate(**encoded_input)
  input_text_en = gr_to_en_tokenizer.batch_decode(output, skip_special_tokens=True)

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
  result = pipe(f"<s>[INST] {input_text_en} [/INST]")
  result = result[0]['generated_text']

  #transformer en to el
  encoded_input = en_to_el_tokenizer(result, return_tensors="pt", padding=True)
  output = en_to_el_model.generate(**encoded_input)
  output_text_el = en_to_el_tokenizer.batch_decode(output, skip_special_tokens=True)
  print('output_text_el: ', output_text_el)
except Exception as e:
  print(e)

input_text_el:  Ποια είναι τα βήματα της διαδικασίας αλλαγής στοιχείων επικοινωνίας αλλοδαπών;
index out of range in self


In [ ]:
input_text_el = "Τι αφορα η διαδικασία έκδοσης δελτίου ανεργίας;"
print('input_text_el: ', input_text_el)

#transformer el to en
encoded_input = gr_to_en_tokenizer(input_text_el, return_tensors="pt", padding=True)
output = gr_to_en_model.generate(**encoded_input)
input_text_en = gr_to_en_tokenizer.batch_decode(output, skip_special_tokens=True)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(f"<s>[INST] {input_text_en} [/INST]")
print('result: ', result)

result = result[0]['generated_text']

print('result after : ', result)

#transformer en to el
encoded_input = en_to_el_tokenizer(result, return_tensors="pt", padding=True)
output = en_to_el_model.generate(**encoded_input)
output_text_el = en_to_el_tokenizer.batch_decode(output, skip_special_tokens=True)
print('output_text_el: ', output_text_el)

input_text_el:  Τι αφορα η διαδικασία έκδοσης δελτίου ανεργίας;
result:  [{'generated_text': "<s>[INST] ['What is the procedure for issuing an unemployment card?'] [/INST]  The procedure for issuing an unemployment card varies depending on the state or country where you are applying for unemployment benefits. everybody. Here are the general steps you can expect to follow:\n\n1. File for unemployment benefits: To get an unemployment card, you first need to file for unemployment benefits with your state's unemployment insurance program. You can usually do this online or in person at your local unemployment office. You will need to provide personal information, such as your name, address, and Social Security number, as well as information about your previous employer and reason for unemployment.\n2. Wait for a decision: After you file for unemployment benefits, the state will review your application and make a decision on whether you are eligible for benefits. This can take several days o

In [4]:
# GET PREDICTIONS AND SAVE REFERENCES AND PREDICTIONS IN AN EXCEL FILE
import openpyxl
import pandas as pd

# Create an Excel workbook and add a worksheet
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = "Predictions"

# Add headers to the worksheet
worksheet.append(["whole_reference", "answer_reference", "answer_prediction"])

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)

# Read the Excel file
df = pd.read_excel('test_data.xlsx')

answer_reference = ''

# Check if column 'text_el' exists
if 'text_el' in df.columns:

    for index, row in df.iterrows():
      if index > 0:
        # print(f"Row {index}")

        prompt = row['text_el']

        # Process each text and text_el as needed
        # print(f"Row {index}: text_el = {prompt}")

        if(prompt != ""):
            prompt = prompt.split("[/INST]")
            if(len(prompt) > 1):
                answer_reference = prompt[1]
                prompt = prompt[0]
                prompt = prompt.split("[INST]")
                if(len(prompt) > 1):
                    prompt_el = prompt[1]

                    try:
                      # Transformer el to en
                      encoded_input = gr_to_en_tokenizer(prompt_el, return_tensors="pt", padding=True)
                      output = gr_to_en_model.generate(**encoded_input)
                      prompt_en = gr_to_en_tokenizer.batch_decode(output, skip_special_tokens=True)

                      result = pipe(f"<s>[INST] {prompt_en} [/INST]")
                      result = result[0]['generated_text']

                      result = result.split("[/INST]")
                      result = result[1]

                      # Transformer en to el
                      encoded_input = en_to_el_tokenizer(result, return_tensors="pt", padding=True)
                      output = en_to_el_model.generate(**encoded_input)
                      result_el = en_to_el_tokenizer.batch_decode(output, skip_special_tokens=True)[0]

                      # Add result to the worksheet
                      worksheet.append([row["text_el"], answer_reference, result_el])
                      workbook.save('reference_prediction.xlsx')

                    except Exception as e:
                      # print("error: ")
                      # print(e)
                      worksheet.append([row["text_el"], answer_reference, ''])
                      workbook.save('reference_prediction.xlsx')

                else:
                    # print("Else 1 - No element : [INST] ")
                    worksheet.append([row["text_el"], answer_reference, ''])
                    workbook.save('reference_prediction.xlsx')
            else:
                # print("Else 2 - No element : [/INST] found")
                worksheet.append([row["text_el"], answer_reference, ''])
                workbook.save('reference_prediction.xlsx')
        else:
            # print("Elese 3 - Row text el empty")
            worksheet.append([row["text_el"], answer_reference, ''])
            workbook.save('whole_reference_answer_reference_prediction.xlsx')

    print("Predictions added and saved to 'reference_prediction.xlsx'")
else:
    print("The required column is not present in the Excel file.")

Predictions added and saved to 'reference_prediction.xlsx'


In [ ]:
# Evaluation of the model -- BLEU
import pandas as pd
from google.colab import drive
import evaluate
bleu = evaluate.load('bleu')

# New instruction dataset
drive.mount('/content/drive')

# Read the Excel file
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/DATASET/references_predictions.xlsx')

print('columns ')
print(df.columns)

# Replace NaN values with empty strings in 'answer_reference' and 'answer_prediction'
df['answer_reference'] = df['answer_reference'].fillna('')
df['answer_prediction'] = df['answer_prediction'].fillna('')

# Extract the 'answer_reference' and 'answer_prediction' columns
references = df['answer_reference'].tolist()
predictions = df['answer_prediction'].tolist()

# BLEU expects the references to be a list of lists
references = [[ref] for ref in references]

# BLEU score
bleu_results = bleu.compute(predictions=predictions, references=references, max_order = 2)

print("bleu_results: ")
print(bleu_results)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
columns 
Index(['whole_reference', 'answer_reference', 'answer_prediction'], dtype='object')
bleu_results: 
{'bleu': 0.02945195662287727, 'precisions': [0.07389573108621168, 0.011738401341531582], 'brevity_penalty': 1.0, 'length_ratio': 3.0105938109841093, 'translation_length': 21598, 'reference_length': 7174}


In [2]:
# GET PREDICTIONS AND SAVE REFERENCES AND PREDICTIONS IN AN EXCEL FILE FOR THE ORIGINAL MODEL
import openpyxl
import pandas as pd

# Create an Excel workbook and add a worksheet
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = "Predictions"

# Add headers to the worksheet
worksheet.append(["whole_reference", "answer_reference", "answer_prediction"])

# LLAMA 2 7B MODEL AS INITIAL MODEL
model = "NousResearch/Llama-2-7b-chat-hf"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)

# Read the Excel file
df = pd.read_excel('test_data.xlsx')

answer_reference = ''

# Check if column 'text_el' exists
if 'text_el' in df.columns:

    for index, row in df.iterrows():
      if index > 0:
        # print(f"Row {index}")

        prompt = row['text_el']

        # Process each text and text_el as needed
        # print(f"Row {index}: text_el = {prompt}")

        if(prompt != ""):
            prompt = prompt.split("[/INST]")
            if(len(prompt) > 1):
                answer_reference = prompt[1]
                prompt = prompt[0]
                prompt = prompt.split("[INST]")
                if(len(prompt) > 1):
                    prompt_el = prompt[1]

                    try:
                      # Transformer el to en
                      encoded_input = gr_to_en_tokenizer(prompt_el, return_tensors="pt", padding=True)
                      output = gr_to_en_model.generate(**encoded_input)
                      prompt_en = gr_to_en_tokenizer.batch_decode(output, skip_special_tokens=True)

                      result = pipe(f"<s>[INST] {prompt_en} [/INST]")
                      result = result[0]['generated_text']

                      result = result.split("[/INST]")
                      result = result[1]

                      # Transformer en to el
                      encoded_input = en_to_el_tokenizer(result, return_tensors="pt", padding=True)
                      output = en_to_el_model.generate(**encoded_input)
                      result_el = en_to_el_tokenizer.batch_decode(output, skip_special_tokens=True)[0]

                      # Add result to the worksheet
                      worksheet.append([row["text_el"], answer_reference, result_el])
                      workbook.save('reference_prediction_initial_model.xlsx')

                    except Exception as e:
                      # print("error: ")
                      # print(e)
                      worksheet.append([row["text_el"], answer_reference, ''])
                      workbook.save('reference_prediction_initial_model.xlsx')

                else:
                    # print("Else 1 - No element : [INST] ")
                    worksheet.append([row["text_el"], answer_reference, ''])
                    workbook.save('reference_prediction_initial_model.xlsx')
            else:
                # print("Else 2 - No element : [/INST] found")
                worksheet.append([row["text_el"], answer_reference, ''])
                workbook.save('reference_prediction_initial_model.xlsx')
        else:
            # print("Else 3 - Row text el empty")
            worksheet.append([row["text_el"], answer_reference, ''])
            workbook.save('reference_prediction_initial_model.xlsx')

    print("Predictions added and saved to 'reference_prediction.xlsx'")
else:
    print("The required column is not present in the Excel file.")

Predictions added and saved to 'reference_prediction.xlsx'


In [17]:
# Evaluation of the model -- ROUGE - of the initial model
import evaluate
import pandas as pd
# from google.colab import drive

rouge = evaluate.load('rouge')

# New instruction dataset
# drive.mount('/content/drive')

# Read the Excel file
df = pd.read_excel('reference_prediction_initial_model.xlsx')
# df = pd.read_excel('test_data.xlsx')

print('columns ')
print(df.columns)

# Replace NaN values with empty strings in 'answer_reference' and 'answer_prediction'
df['answer_reference'] = df['answer_reference'].fillna('')
df['answer_prediction'] = df['answer_prediction'].fillna('')

# Extract the 'answer_reference' and 'answer_prediction' columns
references = df['answer_reference'].tolist()
predictions = df['answer_prediction'].tolist()

# BLEU expects the references to be a list of lists
references = [[ref] for ref in references]


print("references: ")
print(len(references))

print("predictions: ")
print(len(predictions))

original_model_results = rouge.compute(
    predictions=predictions,
    references=references,
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

columns 
Index(['whole_reference', 'answer_reference', 'answer_prediction'], dtype='object')
references: 
188
predictions: 
188
ORIGINAL MODEL:
{'rouge1': 0.06126082479727662, 'rouge2': 0.03259187620889749, 'rougeL': 0.05890128158588734, 'rougeLsum': 0.05913104721380691}


In [ ]:
# Evaluation of the model -- ROUGE - of the finetuned model
import evaluate
import pandas as pd
from google.colab import drive

rouge = evaluate.load('rouge')

# New instruction dataset
drive.mount('/content/drive')

# Read the Excel file
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/DATASET/references_predictions.xlsx')

print('columns ')
print(df.columns)

# Replace NaN values with empty strings in 'answer_reference' and 'answer_prediction'
df['answer_reference'] = df['answer_reference'].fillna('')
df['answer_prediction'] = df['answer_prediction'].fillna('')

# Extract the 'answer_reference' and 'answer_prediction' columns
references = df['answer_reference'].tolist()
predictions = df['answer_prediction'].tolist()

# BLEU expects the references to be a list of lists
references = [[ref] for ref in references]

peft_model_results = rouge.compute(
    predictions=predictions,
    references=references,
    use_aggregator=True,
    use_stemmer=True,
)

print('PEFT MODEL:')
print(peft_model_results)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
columns 
Index(['whole_reference', 'answer_reference', 'answer_prediction'], dtype='object')
PEFT MODEL:
{'rouge1': 0.0615292005359917, 'rouge2': 0.0241001777054908, 'rougeL': 0.06119434391930145, 'rougeLsum': 0.060625528027904926}


In [21]:
import np

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 0.03%
rouge2: -0.85%
rougeL: 0.23%
rougeLsum: 0.15%
